In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [2]:
# Set data directory path
data_dir = "data/data11/4/"

# Load leader and follower density data: shape (n_time, n_space)
rho_l = pd.read_csv(os.path.join(data_dir, "rho_l_values.csv"), header=None).values[:602, :]
rho_f = pd.read_csv(os.path.join(data_dir, "rho_f_values.csv"), header=None).values[:602, :]

# Load time vector: shape (n_time,)
t = pd.read_csv(os.path.join(data_dir, "t_values.csv"))["time"].values[:602].astype(float)

# Construct spatial grid: uniform from 0 to 40, with 101 points
x = np.linspace(0, 40, 101)

# Compute spatial spacing (uniform)
dx = x[1] - x[0]

# No assumption of uniform time spacing — dt will be handled pointwise
dt_array = np.diff(t)

In [3]:
print(f"rho_l shape: {rho_l.shape}")  # (602, 101)
print(f"rho_f shape: {rho_f.shape}")
print(f"t shape: {t.shape}, x shape: {x.shape}")
print(f"dx = {dx:.4f}")
print(f"Is time grid uniform? {np.allclose(np.diff(t), dt_array[0])}")

rho_l shape: (602, 101)
rho_f shape: (602, 101)
t shape: (602,), x shape: (101,)
dx = 0.4000
Is time grid uniform? False


In [4]:
# Parameters for Savitzky-Golay filter (must be odd)
window_length = 9
polyorder = 3

# --- Spatial Derivatives (uniform grid, use Savitzky-Golay) ---
# First derivative: ∂ρ/∂x
rho_l_dx = savgol_filter(rho_l, window_length=window_length, polyorder=polyorder,
                         deriv=1, delta=dx, axis=1)
rho_f_dx = savgol_filter(rho_f, window_length=window_length, polyorder=polyorder,
                         deriv=1, delta=dx, axis=1)

# Second derivative: ∂²ρ/∂x²
rho_l_dxx = savgol_filter(rho_l, window_length=window_length, polyorder=polyorder,
                          deriv=2, delta=dx, axis=1)
rho_f_dxx = savgol_filter(rho_f, window_length=window_length, polyorder=polyorder,
                          deriv=2, delta=dx, axis=1)

In [5]:
# Initialize time derivatives with zeros
rho_l_dt = np.zeros_like(rho_l)
rho_f_dt = np.zeros_like(rho_f)

# Define epsilon threshold to avoid division by zero
epsilon = 1e-12

# Central difference with epsilon protection for interior points
for i in range(1, len(t)-1):
    dt_forward = t[i+1] - t[i]
    dt_backward = t[i] - t[i-1]
    dt_total = dt_forward + dt_backward
    dt_total = dt_total if abs(dt_total) > epsilon else epsilon  # fallback

    rho_l_dt[i, :] = (rho_l[i+1, :] - rho_l[i-1, :]) / dt_total
    rho_f_dt[i, :] = (rho_f[i+1, :] - rho_f[i-1, :]) / dt_total

# Forward difference at the first time point (t=0)
dt0 = max(t[1] - t[0], epsilon)
rho_l_dt[0, :] = (rho_l[1, :] - rho_l[0, :]) / dt0
rho_f_dt[0, :] = (rho_f[1, :] - rho_f[0, :]) / dt0

# Backward difference at the last time point (t=-1)
dt_end = max(t[-1] - t[-2], epsilon)
rho_l_dt[-1, :] = (rho_l[-1, :] - rho_l[-2, :]) / dt_end
rho_f_dt[-1, :] = (rho_f[-1, :] - rho_f[-2, :]) / dt_end

\begin{equation}
\frac{\partial \rho_l}{\partial t}
= + D_l \cdot \frac{\partial^2 \rho_l}{\partial x^2}
- 2\kappa\alpha_l \cdot \rho_l \cdot \frac{\partial^2 \rho_l}{\partial x^2}
- 2\kappa\alpha_l \cdot \left( \frac{\partial \rho_l}{\partial x} \right)^2
- \kappa\alpha_{fl} \cdot \frac{\partial}{\partial x} \left( \rho_l \cdot \frac{\partial \rho_f}{\partial x} \right)
- \chi\beta \cdot \frac{\partial \rho_l}{\partial x}
\end{equation}

\begin{equation}
\frac{\partial \rho_f}{\partial t}
= + D_f \cdot \frac{\partial^2 \rho_f}{\partial x^2}
- 2\kappa\alpha_f \cdot \rho_f \cdot \frac{\partial^2 \rho_f}{\partial x^2}
- 2\kappa\alpha_f \cdot \left( \frac{\partial \rho_f}{\partial x} \right)^2
- \kappa\alpha_{fl} \cdot \frac{\partial}{\partial x} \left( \rho_f \cdot \frac{\partial \rho_l}{\partial x} \right)
\end{equation}

In [6]:
# Composite term: ∂x (rho_l * ∂rho_f/∂x)
rho_l_grad_rho_f = rho_l * rho_f_dx
d_rho_l_grad_rho_f_dx = savgol_filter(rho_l_grad_rho_f, window_length=window_length, polyorder=polyorder,
                                      deriv=1, delta=dx, axis=1)

# Dictionary for leader PDE
theta_leader = np.stack([
    rho_l_dxx.ravel(),                         # D_l
    (rho_l * rho_l_dxx).ravel(),               # -2κ α_l (ρ_l ρ_l'')
    (rho_l_dx ** 2).ravel(),                   # -2κ α_l (ρ_l')^2
    d_rho_l_grad_rho_f_dx.ravel(),             # -κ α_fl (ρ_l ∂ρ_f)
    rho_l_dx.ravel()                           # -χβ ∂ρ_l
], axis=1)

# Target
target_leader = rho_l_dt.ravel()

In [7]:
# Composite term: ∂x (rho_f * ∂rho_l/∂x)
rho_f_grad_rho_l = rho_f * rho_l_dx
d_rho_f_grad_rho_l_dx = savgol_filter(rho_f_grad_rho_l, window_length=window_length, polyorder=polyorder,
                                      deriv=1, delta=dx, axis=1)

# Construct dictionary matrix for follower PDE
theta_follower = np.stack([
    rho_f_dxx.ravel(),                        # θ₁: ∂²ρ_f/∂x²
    (rho_f * rho_f_dxx).ravel(),              # θ₂: ρ_f ⋅ ∂²ρ_f/∂x²
    (rho_f_dx ** 2).ravel(),                  # θ₃: (∂ρ_f/∂x)²
    d_rho_f_grad_rho_l_dx.ravel()             # θ₄: ∂x(ρ_f ⋅ ∂ρ_l/∂x)
], axis=1)

# Target variable: ∂ρ_f/∂t
target_follower = rho_f_dt.ravel()

In [8]:
# 1. Define feature names for leader PDE
param_names_leader = [
    "D_l",
    "-2 * kappa * alpha_l (rho_l ⋅ rho_l'')",
    "-2 * kappa * alpha_l ((rho_l')^2)",
    "-kappa * alpha_fl (rho_l ⋅ rho_f')",
    "-chi * beta (rho_l')"
]

# 2. Standardize features
scaler = StandardScaler()
theta_l_scaled = scaler.fit_transform(theta_leader)

# 3. Remove invalid rows
mask = np.isfinite(theta_l_scaled).all(axis=1) & np.isfinite(target_leader)
X_l_clean = theta_l_scaled[mask]
y_l_clean = target_leader[mask]

# 4. Define models to try
models = {
    "Ridge (alpha=1e3)": Ridge(alpha=1e3),
    "Lasso (alpha=1e-2)": Lasso(alpha=1e-2, max_iter=10000),
    "ElasticNet (alpha=1e-2, l1_ratio=0.5)": ElasticNet(alpha=1e-2, l1_ratio=0.5, max_iter=10000),
    "HuberRegressor": HuberRegressor()
}

# 5. Fit and report each model
for model_name, model in models.items():
    model.fit(X_l_clean, y_l_clean)
    coefs = model.coef_
    preds = model.predict(X_l_clean)
    mse = mean_squared_error(y_l_clean, preds)

    print(f"\n Recovered parameter combinations using {model_name} (Leader PDE):")
    for name, value in zip(param_names_leader, coefs):
        print(f"{name:55s}: {value:.6f}")
    print(f"Mean Squared Error: {mse:.6e}")


 Recovered parameter combinations using Ridge (alpha=1e3) (Leader PDE):
D_l                                                    : -359050.633622
-2 * kappa * alpha_l (rho_l ⋅ rho_l'')                 : 204634.622070
-2 * kappa * alpha_l ((rho_l')^2)                      : -183122.513032
-kappa * alpha_fl (rho_l ⋅ rho_f')                     : 125573.143000
-chi * beta (rho_l')                                   : 26206.520388
Mean Squared Error: 2.237265e+14

 Recovered parameter combinations using Lasso (alpha=1e-2) (Leader PDE):
D_l                                                    : -456447.533788
-2 * kappa * alpha_l (rho_l ⋅ rho_l'')                 : 302174.352609
-2 * kappa * alpha_l ((rho_l')^2)                      : -178906.242625
-kappa * alpha_fl (rho_l ⋅ rho_f')                     : 125520.542303
-chi * beta (rho_l')                                   : 23511.175853
Mean Squared Error: 2.237256e+14

 Recovered parameter combinations using ElasticNet (alpha=1e-2, l1_ratio=0

In [9]:
# (1) Feature names for display
param_names_follower = [
    "D_f",
    "-2 * kappa * alpha_f (rho_f ⋅ rho_f'')",
    "-2 * kappa * alpha_f ((rho_f')^2)",
    "-kappa * alpha_fl (rho_f ⋅ rho_l')"
]

# (2) Clean and standardize features
scaler = StandardScaler()
theta_f_scaled = scaler.fit_transform(theta_follower)
mask = np.isfinite(theta_f_scaled).all(axis=1) & np.isfinite(target_follower)
X_clean = theta_f_scaled[mask]
y_clean = target_follower[mask]

# (3) Define models to compare
models = {
    "Ridge (alpha=1e3)": Ridge(alpha=1e3),
    "Lasso (alpha=1e-2)": Lasso(alpha=1e-2, max_iter=10000),
    "ElasticNet (alpha=1e-2, l1_ratio=0.5)": ElasticNet(alpha=1e-2, l1_ratio=0.5, max_iter=10000),
    "HuberRegressor": HuberRegressor()
}

# (4) Fit and report each model
for model_name, model in models.items():
    model.fit(X_clean, y_clean)
    coefs = model.coef_
    preds = model.predict(X_clean)
    mse = mean_squared_error(y_clean, preds)

    print(f"\n Recovered parameter combinations using {model_name} (Follower PDE):")
    for name, value in zip(param_names_follower, coefs):
        print(f"{name:55s}: {value:.6f}")
    print(f"Mean Squared Error: {mse:.6e}")


 Recovered parameter combinations using Ridge (alpha=1e3) (Follower PDE):
D_f                                                    : 4469331.242261
-2 * kappa * alpha_f (rho_f ⋅ rho_f'')                 : -47392618.656660
-2 * kappa * alpha_f ((rho_f')^2)                      : 9623230.208531
-kappa * alpha_fl (rho_f ⋅ rho_l')                     : 54513937.939033
Mean Squared Error: 5.157118e+15

 Recovered parameter combinations using Lasso (alpha=1e-2) (Follower PDE):
D_f                                                    : 1103989.895665
-2 * kappa * alpha_f (rho_f ⋅ rho_f'')                 : -73864704.368089
-2 * kappa * alpha_f ((rho_f')^2)                      : 19337570.169272
-kappa * alpha_fl (rho_f ⋅ rho_l')                     : 88242305.613717
Mean Squared Error: 5.104953e+15

 Recovered parameter combinations using ElasticNet (alpha=1e-2, l1_ratio=0.5) (Follower PDE):
D_f                                                    : 2538181.536617
-2 * kappa * alpha_f (rho_f ⋅ rho